# Minimum Eigen Optimizer


## About this notebook and the adaptation to Amazon Braket backend
The original version of this notebook is accessible at [docs/tutorials/03_minimum_eigen_optimizer.ipynb](https://github.com/qiskit-community/qiskit-optimization/blob/stable/0.6/docs/tutorials/03_minimum_eigen_optimizer.ipynb). The only changes made to the original notebook are contained in the first cell and in cell #6 where we pass the `BackendSampler` object to execute the tasks on Amazon Braket SV1 simulator.

In [1]:
from qiskit.primitives import BackendSamplerV2

from qiskit_braket_provider import BraketProvider

sampler = BackendSamplerV2(
    backend=BraketProvider().get_backend("SV1"),
)

## Introduction

An interesting class of optimization problems to be addressed by quantum computing are Quadratic Unconstrained Binary Optimization (QUBO) problems.
Finding the solution to a QUBO is equivalent to finding the ground state of a corresponding Ising Hamiltonian, which is an important problem not only in optimization, but also in quantum chemistry and physics. For this translation, the binary variables taking values in $\{0, 1\}$ are replaced by spin variables taking values in $\{-1, +1\}$, which allows one to replace the resulting spin variables by Pauli Z matrices, and thus, an Ising Hamiltonian. For more details on this mapping we refer to [1].

Qiskit optimization provides automatic conversion from a suitable `QuadraticProgram` to an Ising Hamiltonian, which then allows leveraging all the `SamplingMinimumEigensolver` implementations, such as

- `SamplingVQE`,
- `QAOA`, or
- `NumpyMinimumEigensolver` (classical exact method).

Note 1: `MinimumEigenOptimizer` does not support `qiskit_algorithms.VQE`. But `qiskit_algorithms.SamplingVQE`
can be used instead.

Note 2: `MinimumEigenOptimizer` can use `NumpyMinimumEigensolver` as an exception case though it inherits `MinimumEigensolver` (not `SamplingMinimumEigensolver`).

Qiskit optimization provides a the `MinimumEigenOptimizer` class, which wraps the translation to an Ising Hamiltonian (in Qiskit Terra also called `SparsePauliOp`), the call to a `MinimumEigensolver`, and the translation of the results back to an `OptimizationResult`.

In the following we first illustrate the conversion from a `QuadraticProgram` to a `SparsePauliOp` and then show how to use the `MinimumEigenOptimizer` with different `MinimumEigensolver`s to solve a given `QuadraticProgram`.
The algorithms in Qiskit optimization automatically try to convert a given problem to the supported problem class if possible, for instance, the `MinimumEigenOptimizer` will automatically translate integer variables to binary variables or add linear equality constraints as a quadratic penalty term to the objective. It should be mentioned that a `QiskitOptimizationError` will be thrown if conversion of a quadratic program with integer variables is attempted.

The circuit depth of `QAOA` potentially has to be increased with the problem size, which might be prohibitive for near-term quantum devices.
A possible workaround is Recursive QAOA, as introduced in [2].
Qiskit optimization generalizes this concept to the `RecursiveMinimumEigenOptimizer`, which is introduced at the end of this tutorial.

### References
[1] [A. Lucas, *Ising formulations of many NP problems,* Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, *Obstacles to State Preparation and Variational Optimization from Symmetry Protection,* arXiv  preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)

## Converting a QUBO to a SparsePauliOp

In [2]:
from typing import List, Tuple

import numpy as np
from qiskit.visualization import plot_histogram
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    OptimizationResultStatus,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
)
from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.optimizers import COBYLA
from qiskit_optimization.utils import algorithm_globals


In [3]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var("x")
qubo.binary_var("y")
qubo.binary_var("z")
qubo.minimize(linear=[1, -2, 3], quadratic={("x", "y"): 1, ("x", "z"): -1, ("y", "z"): 2})
print(qubo.prettyprint())

Problem name: 

Minimize
  x*y - x*z + 2*y*z + x - 2*y + 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



Next we translate this QUBO into an Ising operator. This results not only in a `SparsePauliOp` but also in a constant offset to be taken into account to shift the resulting value.

In [4]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)

offset: 1.5
operator:
SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[-0.5 +0.j,  0.25+0.j, -1.75+0.j,  0.25+0.j, -0.25+0.j,  0.5 +0.j])


Sometimes a `QuadraticProgram` might also directly be given in the form of a `SparsePauliOp`. For such cases, Qiskit optimization also provides a translator from a `SparsePauliOp` back to a `QuadraticProgram`, which we illustrate in the following.

In [5]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.prettyprint())

Problem name: 

Minimize
  x0*x1 - x0*x2 + 2*x1*x2 + x0 - 2*x1 + 3*x2

Subject to
  No constraints

  Binary variables (3)
    x0 x1 x2



This translator allows, for instance, one to translate a `SparsePauliOp` to a `QuadraticProgram` and then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, such as the `GroverOptimizer`.

## Solving a QUBO with the MinimumEigenOptimizer

We start by initializing the `MinimumEigensolver` we want to use.

In [6]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=sampler, optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

/Users/sesmart/.local/share/mise/installs/python/3.12.10/lib/python3.12/site-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


Then, we use the `MinimumEigensolver` to create `MinimumEigenOptimizer`.

In [7]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

We first use the `MinimumEigenOptimizer` based on the classical exact `NumPyMinimumEigensolver` to get the optimal benchmark solution for this small example.

In [8]:
exact_result = exact.solve(qubo)
print(exact_result.prettyprint())

objective function value: -2.0
variable values: x=0.0, y=1.0, z=0.0
status: SUCCESS


Next we apply the `MinimumEigenOptimizer` based on `QAOA` to the same problem.

In [9]:
qaoa

In [10]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint()) 

/Users/sesmart/coding/development/qiskit-braket-provider/qiskit_braket_provider/providers/adapter.py:617: UserWarning: The Qiskit circuit contains barrier instructions that are ignored.
  warnings.warn("The Qiskit circuit contains barrier instructions that are ignored.")
/Users/sesmart/.local/share/mise/installs/python/3.12.10/lib/python3.12/site-packages/braket/aws/aws_quantum_task.py:774: UserWarning: AwsQuantumTask.__init__ received unknown keyword args: ['memory', 'seed_simulator']
  return AwsQuantumTask(task_arn, aws_session, *args, **kwargs)


None
1
[_MeasureInfo(creg_name='meas', num_bits=3, num_bytes=1, start=0)]
['101', '010', '110', '010', '010', '000', '011', '110', '001', '011', '100', '101', '011', '000', '100', '000', '101', '110', '010', '101', '001', '000', '001', '110', '001', '101', '001', '000', '100', '110', '011', '101', '100', '000', '111', '100', '101', '111', '011', '101', '100', '001', '011', '001', '000', '111', '111', '101', '000', '011', '010', '001', '001', '011', '110', '111', '111', '111', '111', '100', '011', '010', '001', '101', '111', '011', '111', '011', '011', '111', '001', '001', '011', '101', '111', '011', '011', '001', '011', '110', '111', '101', '101', '011', '011', '111', '100', '110', '100', '011', '110', '000', '101', '010', '001', '011', '100', '011', '101', '100', '111', '000', '110', '011', '101', '100', '010', '001', '101', '001', '111', '011', '011', '010', '000', '100', '111', '110', '010', '001', '000', '111', '011', '001', '110', '110', '010', '011', '011', '100', '100', '101', '

OverflowError: int too big to convert

### Analysis of Samples
`OptimizationResult` provides useful information in the form of `SolutionSample`s (here denoted as *samples*). Each `SolutionSample` contains
information about the input values (`x`), the corresponding objective function value (`fval`), the fraction of samples corresponding to that input (`probability`),
and the solution `status` (`SUCCESS`, `FAILURE`, `INFEASIBLE`). Multiple samples corresponding to the same input are consolidated into a single `SolutionSample` (with its `probability` attribute being the aggregate fraction of samples represented by that `SolutionSample`).

In [ ]:
print("variable order:", [var.name for var in qaoa_result.variables])
for s in qaoa_result.samples:
    print(s)

We may also want to filter samples according to their status or probabilities.

In [ ]:
def get_filtered_samples(
    samples: List[SolutionSample],
    threshold: float = 0,
    allowed_status: Tuple[OptimizationResultStatus] = (OptimizationResultStatus.SUCCESS,),
):
    return [s for s in samples if s.status in allowed_status and s.probability > threshold]

In [ ]:
filtered_samples = get_filtered_samples(
    qaoa_result.samples,
    threshold=0.005,
    allowed_status=(OptimizationResultStatus.SUCCESS,),
)
for s in filtered_samples:
    print(s)

If we want to obtain a better perspective of the results, statistics is very helpful, both with respect to
the objective function values and their respective probabilities. Thus, mean and standard deviation are the very
basics for understanding the results.

In [ ]:
fvals = [s.fval for s in qaoa_result.samples]
probabilities = [s.probability for s in qaoa_result.samples]

In [ ]:
np.mean(fvals)

In [ ]:
np.std(fvals)

Finally, despite all the number-crunching, visualization is usually the best early-analysis approach.

In [ ]:
samples_for_plot = {
    " ".join(f"{qaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x)): s.probability
    for s in filtered_samples
}
samples_for_plot

In [ ]:
plot_histogram(samples_for_plot)

## RecursiveMinimumEigenOptimizer

The `RecursiveMinimumEigenOptimizer` takes a `MinimumEigenOptimizer` as input and applies the recursive optimization scheme to reduce the size of the problem one variable at a time.
Once the size of the generated intermediate problem is below a given threshold (`min_num_vars`), the `RecursiveMinimumEigenOptimizer` uses another solver (`min_num_vars_optimizer`), e.g., an exact classical solver such as CPLEX or the `MinimumEigenOptimizer` based on the `NumPyMinimumEigensolver`.

In the following, we show how to use the `RecursiveMinimumEigenOptimizer` using the two `MinimumEigenOptimizer`s introduced before.

First, we construct the `RecursiveMinimumEigenOptimizer` such that it reduces the problem size from 3 variables to 1 variable and then uses the exact solver for the last variable. Then we call `solve` to optimize the considered problem.

In [ ]:
rqaoa = RecursiveMinimumEigenOptimizer(qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

In [ ]:
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result.prettyprint())

In [ ]:
filtered_samples = get_filtered_samples(
    rqaoa_result.samples,
    threshold=0.005,
    allowed_status=(OptimizationResultStatus.SUCCESS,),
)

In [ ]:
samples_for_plot = {
    " ".join(f"{rqaoa_result.variables[i].name}={int(v)}" for i, v in enumerate(s.x)): s.probability
    for s in filtered_samples
}
samples_for_plot

In [ ]:
plot_histogram(samples_for_plot)